In [1]:
'''Clear out temporary variables and get a clean session'''
try:
    from IPython import get_ipython
    get_ipython().magic('clear')
    get_ipython().magic('reset -f')
except:
    pass

### Data preparation
The data in this dataset come from a real-world admissions board for a large high school district.  The district runs several specialized programs for which they run admissions.  As such, the data is taken from spreadsheets created over the course of 10 years.  As such, column names, admissions criteria used and data maintained over the years has changed. This initial data preparation involves getting the original data into a consistent format.  There will be no data manipulation or analysis conducted at this stage.

In [2]:
'''Import necessary libraries'''
import pandas as pd
import numpy as np
import os
import glob
import re

import missingno as msno
import pickle
import bz2

import sklearn
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import set_config
#from sklearn import missingpy

%matplotlib inline

In [ ]:
'''Import all csv files in active folder and create dictionary of dataframes'''
csv_files = glob.glob('*.csv')

var_dict = {}

for file in csv_files:
    key = 'df_'+str(re.split('.csv', file)[0])
    value = pd.read_csv(file)
    var_dict[key] = value

In [ ]:
'''Update global variable table and create individual dataframes'''
'''globals().update(var_dict)

df_list = []
for key, value in var_dict.items():
    print(key,value)'''

In [ ]:
'''List of all dataframes'''
var_dict.keys()

In [ ]:
var_dict['df_2008_2009_ib_magnet'].columns

In [ ]:
'''List of columns to drop from original files'''
drop_columns = ['Record Created Date', '#', 'Sibling', 'NYr Grade', 'Also GHS', 'Tier', 'DOB', 'Address', 'City', 'State',\
                'ZIP', 'Home Phone', 'Work/Cell Phone', 'Parent email', 'ID', 'District of Residence', 'Sending District',\
               'Current School', 'Middle School', 'Current School', 'Current Grade', 'Grade Level', 'OHS', 'Academy?',\
               'Academy', 'Math Course', 'Gender', 'G', 'Foreign Language', 'Length', 'IEP?', 'IEP/ISP', 'IEP', '504?',\
               '504', '7th Grade Math', '7th Grade Sci', '7th Grade Eng', '7th Grade SS', '7th Grade WL', '8th Grade Math',\
               '8th Grade Sci', '8th Grade Eng', '8th Grade SS', '8th Grade WL', 'Transcripts', 'Transcript', 'Transcript?',\
               'Attendance?', 'Attendance', 'Discipline?', 'Discipline', 'Tests?', 'Recommendations', 'Recs', 'Rec',\
               'Notes', 'Notes', 'Notes.1', 'Notes.2' 'Math/Sci Rec', 'Math/Sci Adv Course', 'Eng/SS Adv Course', 'Adv Courses',\
               'Waitlist #', 'WL #', 'Withdrew  in first year', 'Withdrew', 'Reason', 'Acc Sp.', 'Acc Sp Gr', 'GPA 1st year',\
               'Prediction', 'Student email', 'Requested HS', 'Sending Supt.', 'Preference', 'Info Release consent',\
               'Tuition Code', 'Reception Letter Sent', 'Reception RSVP', 'Testing Number', 'Ethnic', 'Unnamed: 49', \
               'Unnamed: 53', 'Unnamed: 52', 'Prospect Street', 'Prospect City', 'Prospect State', 'Prospect Zip', \
               'Resident District', 'Filler', 'MCST Academy', 'P.I.Score', 'PI Score', 'AX Offer?.1', 'Car Pooling info',\
               'Unnamed: 59', 'Unnamed: 60', 'Eng/SS Rec', 'Math/Sci Rec', 'Math/Sci Adv', 'Eng/SS Adv', 'Adv. Courses', \
               'Parent/Guardian Name']

drop_columns = [column.lower() for column in drop_columns]

In [ ]:
'''Drop unnecessary columns from original data'''
for key, item in var_dict.items():
    for column in item.columns:
        column_lower = column.strip().lower()
        if column_lower in drop_columns:
            item.drop(column, axis=1, inplace = True)

In [ ]:
var_dict['df_2008_2009_ib_magnet'].columns

In [ ]:
'''Create list of all columns'''
column_list = []
for key,item in var_dict.items():
    column_list.append(item.columns)
column_list

In [ ]:
'''Dictionary to standardize column names'''
rename_dict = {'CL': 'program',
               'Program': 'program',
               'L.N': 'last_name', 'Last Name': 'last_name',
               'F.N': 'first_name', 'First Name': 'first_name',
               'ES': 'essay_raw', 'WP': 'essay_raw', 'ACC Score Raw': 'essay_raw',
               'ESC': 'essay_converted', 'WPC': 'essay_converted', 'ACC C': 'essay_converted', 
               'ESW': 'essay_weighted', 'WPW': 'essay_weighted', 'ACC W': 'essay_weighted',
               'ME': 'math_raw', 'AR': 'math_raw',
               'MEC': 'math_converted', 'ARC': 'math_converted', 
               'MEW': 'math_weighted', 'ARW': 'math_weighted', 
               'GP': 'gpa_raw', 'GPA': 'gpa_raw',
               'GPC': 'gpa_converted', 'GPAC': 'gpa_converted',
               'GPW': 'gpa_weighted', 'GPAW': 'gpa_weighted', 
               'STE': 'standard_ela',
               'STEC': 'standard_ela_converted',
               'STEW': 'standard_ela_weighted',
               'STM':  'standard_math',
               'TOTAL': 'standard_total',  
               'STC': 'standard_total_converted', 
               'STW': 'standard_total_weighted',
               'RD': 'reading_score_raw',
               'RDC': 'reading_score_converted',
               'RDW': 'reading_score_weighted',
               'EA': 'qas', 'QAS': 'qas',
               'EAC': 'qas_converted', 'QASC': 'qas_converted',
               'EAW': 'qas_weighted', 'QASW': 'qas_weighted',         
               'RC': 'recommend', 'Rec': 'recommend', 
               'RCC': 'recommend_converted', 
               'RCW': 'recommend_weighted', 'RecW': 'recommend_weighted', 'RECW': 'recommend_weighted',
               'IN': 'interview', 
               'INC': 'interview_converted', 
               'INW' : 'interview_weighted', 
               'Admit Score': 'admit_score', 'Total': 'admit_score',
               'Decision': 'admit_decision', 'Admit Status': 'admit_decision', 'Accepted Offer?': 'admit_decision', 'Status': 'admit_decision',
               'Accepted Offer?': "admit_offer_accepted", 'AX Offer?': "admit_offer_accepted", 'Attending?': "admit_offer_accepted"
}

In [ ]:
'''Standardize Column Names'''
drop_names = ['last_name', 'first_name', 'Name']

for key,item in var_dict.items():
    item.rename(columns = rename_dict, inplace = True)
    for column in item.columns:
        if column in drop_names:
            item.drop(column, axis = 1, inplace = True)
    print(key,":", item.columns)

In [ ]:
'''Find unique values in each column'''
unique_check = ['program', 'admit_decision', 'admit_offer_accepted']

for key,item in var_dict.items():
    print("\n",key)
    for column in item:
        if column in unique_check:
            print(column, item[column].unique())

In [ ]:
'''Drop empty and non-IB/magnet rows'''
for key,item in var_dict.items():
    item.dropna(axis = 0, thresh = 12, inplace = True)
    item.dropna(axis = 0, subset = ['program'], inplace=True)
    item = item[item['program'] != 'Regular']
    print(key, item)

In [ ]:
'''Correct program in certain dataframes'''
ib_list = ['df_2009_2010_ib', 'df_2010-2011_ib']
magnet_list = ['df_2009_2010_magnet', 'df_2010-2011_magnet']

for key,item in var_dict.items():
    if key in ib_list:
        item['program'] = 'IB'
    if key in magnet_list:
        item['program'] = 'Magnet'

In [ ]:
'''Find unique values in each column'''
unique_check = ['program', 'admit_decision', 'admit_offer_accepted']

for key,item in var_dict.items():
    print("\n",key)
    for column in item:
        if column in unique_check:
            print(column, item[column].unique())

In [ ]:
'''Standardize data'''
for key,item in var_dict.items():
    item['admit_decision'] = item['admit_decision'].apply(lambda x: 'ax' if 'ax' in str(x).lower() else 'rj'if ('rj' or 'ret' or 'df') in \
                                                          str(x).lower() else 'wd' if ('wd'or'w/d') in str(x).lower() else 'rj')
    item['admit_offer_accepted'] = item['admit_offer_accepted'].apply(lambda x: 'yes' if 'y' in str(x).lower() else 'no'if ('no' or 'n') in \
                                                          str(x).lower() else 'no')

In [ ]:
'''Print cleaned and standardized dataframes'''
for key,item in var_dict.items():
    print(key, item)

In [ ]:
'''Pickle datafile'''

with open('mib_cleaned_data.pkl', 'wb') as f:
    pickle.dump(var_dict, f)

In [15]:
'''Open pickled file'''
with open('mib_cleaned_data.pkl', 'rb') as f:
    var_dict = pickle.load(f)

### Data cleaning
At this point, the data is standardized and contains only the observations of interest.  It is now time to clean the data.  This will involve dealing with NaN values and standardizing the data.

In [16]:
'''Create list with only numeric variables'''
def select_numeric (df):
    return df.select_dtypes(include="number").columns.tolist()

In [17]:
'''Create list with only object variables'''
def select_object (df):
    return df.select_dtypes(include="object").columns.tolist()

In [18]:
'''Create list with only categorical variables'''
def select_categorical (df):
    return df.select_dtypes(include="category").columns.tolist()

In [19]:
'''Check for missing data'''
for key,item in var_dict.items():
    num_cols = select_numeric(item)
    item[num_cols] = item[num_cols].astype('float64')
    cat_cols = select_object(item)
    item[cat_cols] = item[cat_cols].astype('category')
    print(key, item.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   program                   88 non-null     category
 1   essay_raw                 86 non-null     float64 
 2   essay_converted           88 non-null     float64 
 3   essay_weighted            88 non-null     float64 
 4   math_raw                  86 non-null     float64 
 5   math_converted            88 non-null     float64 
 6   math_weighted             88 non-null     float64 
 7   gpa_raw                   88 non-null     float64 
 8   gpa_converted             88 non-null     float64 
 9   gpa_weighted              88 non-null     float64 
 10  standard_ela              84 non-null     float64 
 11  standard_math             84 non-null     float64 
 12  standard_total            85 non-null     float64 
 13  standard_total_converted  88 non-null     float64 
 

ValueError: Columns must be same length as key

In [ ]:
'''Visualize missing data'''
for key,item in var_dict.items():
    msno.matrix(item.sort_values('admit_decision'))

Given the low rate and nature of missingness, we are going to utilize MissForest imputation to fill missing values.  However, in order to prevent data leakage, this will be conducted within a pipeline when we are ready to build our model.

### In the following cells, the missing data in each dataframe will be imputed and standardized separately. This will be done since different dependent measures were utilized across years.  Each year will be imputed and standarized separately and then the dataframes will be concatanated.

In [8]:
df = (var_dict['df_2008_2009_ib_magnet'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   program                   88 non-null     category
 1   essay_raw                 86 non-null     float64 
 2   essay_converted           88 non-null     float64 
 3   essay_weighted            88 non-null     float64 
 4   math_raw                  86 non-null     float64 
 5   math_converted            88 non-null     float64 
 6   math_weighted             88 non-null     float64 
 7   gpa_raw                   88 non-null     float64 
 8   gpa_converted             88 non-null     float64 
 9   gpa_weighted              88 non-null     float64 
 10  standard_ela              84 non-null     float64 
 11  standard_math             84 non-null     float64 
 12  standard_total            85 non-null     float64 
 13  standard_total_converted  88 non-null     float64 
 

In [29]:
num_cols = select_numeric(df)
print(num_cols)
cat_cols = select_categorical(df)
cat_cols.remove('program')
print(cat_cols)

['essay_raw', 'essay_converted', 'essay_weighted', 'math_raw', 'math_converted', 'math_weighted', 'gpa_raw', 'gpa_converted', 'gpa_weighted', 'standard_ela', 'standard_math', 'standard_total', 'standard_total_converted', 'standard_total_weighted', 'recommend', 'recommend_converted', 'recommend_weighted', 'PIScore', 'interview', 'interview_converted', 'interview_weighted', 'admit_score']
['admit_decision', 'admit_offer_accepted']


In [30]:
'''Transformer for numerical features'''
num_pipe = Pipeline(
    [
        ('imputer_num', KNNImputer()),
        ('scaler', StandardScaler())
    ]
)

In [31]:
'''Transformer for categorical features'''
cat_pipe = Pipeline(
    [
        ('imputer_cat', SimpleImputer(strategy = 'constant', fill_value = 'missing')),
        ('one_hot', OneHotEncoder(handle_unknown='ignore', sparse=False)) 
    ]
)

In [32]:
'''Combine transformers into ColumnTransformer'''
preprocessor = ColumnTransformer(
    [
        ('categorical', cat_pipe, cat_cols),
        ('numerical', num_pipe, num_cols)
    ]
)

In [13]:
'''Function to split training and testing datasets'''
X = df.drop('program', axis = 1)
y = df['program']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

'''Fit Model'''
preprocessor.fit(X)

'''Prepare column names'''
cat_columns = preprocessor.named_transformers_['categorical']['one_hot'].get_feature_names_out(cat_cols)
columns = np.append(cat_columns, num_cols)

df_trans = pd.DataFrame(preprocessor.transform(X), columns = columns)
df_trans

,admit_decision_ax,admit_decision_rj,admit_decision_wd,admit_offer_accepted_no,admit_offer_accepted_yes,essay_raw,essay_converted,essay_weighted,math_raw,math_converted,...,standard_total,standard_total_converted,standard_total_weighted,recommend,recommend_converted,recommend_weighted,interview,interview_converted,interview_weighted,admit_score
0,0.0,1.0,0.0,1.0,0.0,-0.099089,0.044269,-1.176804,-0.780948,-1.247101,...,-0.223847,0.227305,0.227305,-0.509563,-0.509563,-0.982368,-0.749739,-2.084892,-1.513810,-1.486848
1,0.0,1.0,0.0,1.0,0.0,0.708301,0.398420,-1.007128,-0.385228,-1.247101,...,1.039632,0.753694,0.753694,-0.358480,-0.358480,-0.887239,-0.566105,-2.084892,-1.513810,-1.570076
2,0.0,0.0,1.0,1.0,0.0,0.708301,0.398420,-1.007128,1.901155,1.403742,...,0.465323,0.227305,0.227305,0.925730,0.925730,-0.078643,-1.711631,-2.084892,-1.513810,0.952382
3,1.0,0.0,0.0,0.0,1.0,0.708301,0.398420,-1.007128,1.769249,1.403742,...,1.200438,0.753694,0.753694,0.850188,0.850188,-0.126208,-2.183833,-0.595683,-1.306277,1.195665
4,1.0,0.0,0.0,0.0,1.0,-0.906478,-0.309882,-1.346479,1.065746,0.873573,...,0.396406,0.227305,0.227305,0.472479,0.472479,-0.364030,-0.609828,0.297842,-1.181758,0.529838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1.0,0.0,0.0,0.0,1.0,2.323080,1.815025,-0.328425,1.241622,1.138658,...,1.384217,0.753694,0.753694,0.774646,0.774646,-0.173772,-2.446167,-0.595683,-1.306277,0.939578
84,1.0,0.0,0.0,1.0,0.0,-0.099089,0.044269,0.519953,0.362244,0.608489,...,-0.568432,-0.299085,-0.299085,0.094771,0.094771,0.420783,-0.522383,0.297842,0.561518,0.299360
85,1.0,0.0,0.0,0.0,1.0,-0.099089,0.044269,-1.176804,0.362244,0.608489,...,-0.568432,-0.299085,-0.299085,0.094771,0.094771,-0.601852,-0.522383,0.297842,-1.181758,0.164914
86,1.0,0.0,0.0,1.0,0.0,-0.099089,0.044269,0.519953,0.406213,0.608489,...,0.809908,0.227305,0.227305,-0.056313,-0.056313,0.301872,1.401401,0.893525,1.080350,0.523436


In [ ]:
'''Pickle datafile'''

with open('mib_transformed_data.pkl', 'wb') as f:
    pickle.dump(var_dict, f)

In [28]:
df = (var_dict['df_2009_2010_ib'])

In [38]:
'''Function to split training and testing datasets'''
X = df.drop('program', axis = 1)
y = df['program']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

'''Fit Model'''
preprocessor.fit(X)

'''Prepare column names'''
cat_columns = preprocessor.named_transformers_['categorical']['one_hot'].get_feature_names_out(cat_cols)
columns = np.append(cat_columns, num_cols)

df_trans = pd.DataFrame(preprocessor.transform(X), columns = columns)
df_trans.head(1)

,admit_decision_ax,admit_decision_rj,admit_offer_accepted_no,admit_offer_accepted_yes,essay_raw,essay_converted,essay_weighted,math_raw,math_converted,math_weighted,...,standard_total_converted,standard_total_weighted,recommend,recommend_converted,recommend_weighted,PIScore,interview,interview_converted,interview_weighted,admit_score
0,1.0,0.0,0.0,1.0,-0.644898,-0.334752,-0.267803,-1.110808,-1.253265,-1.253265,...,0.344942,0.344942,-0.537907,-0.59366,-0.550026,-0.877388,-0.910211,-0.470032,-0.470032,-0.848862


In [ ]:
[ 1.          0.          0.          1.         -0.64489831 -0.33475211
 -0.26780265 -1.11080785 -1.25326474 -1.25326474 -2.20948546 -2.25542823
 -2.25542823  0.43522461  0.27700071  0.36199743  0.34494179  0.34494179
 -0.5379065  -0.59365959 -0.55002597 -0.91021132 -0.47003216 -0.47003216
 -0.84886232]

### Combining all dataframes into a single dataframe

In [ ]:
column_list = var_dict['df_2008_2009_ib_magnet'].columns.tolist()
for key, item in var_dict.items():
    for column in item.columns:
        if column not in column_list:
            column_list.append(column)
print(column_list)

for key, item in var_dict.items():
    for column in item.columns:
        if item in intersection(item.columns, column_list):
            item[column] = np.nan
    

In [ ]:
for key, item in var_dict.items():
    print(len(item.columns))

In [ ]:
'''Perform KNN imputation'''
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(X)
Xtrans = imputer.transform(X)